In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import svt
from bayes_opt import BayesianOptimization

## Example of using singular value thresholding with bayesian optimization

In [2]:
X = np.random.random([5,5])
mask = np.ones_like(X)
nan = [[0,2],[1,2],[3,0],[4,4]]
for idx in nan: mask[idx[0],idx[1]] = 0
mask

array([[1., 1., 0., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0.]])

the bayes_opt packages maximizes by default, so we flip the sign of the norm error

In [3]:
def svt_solve(threshold, eps):
    X_hat = svt.svt_solve(X, mask, threshold=threshold, eps=eps)
    return  - np.linalg.norm(X - X_hat)

there's probably better hyperparamter ranges...

In [4]:
S = np.linalg.svd(X)[1]
pbounds = {"threshold": (S.min(), S.max()),
    "eps": (1e-8, 1e-2)}

In [10]:
optimizer = BayesianOptimization(
    f= svt_solve ,
    pbounds=pbounds,
    verbose=0, 
    random_state=1,
)
optimizer.maximize(
    init_points=0,
    n_iter=100,
)
optimizer.max

{'target': -0.498402894801359,
 'params': {'eps': 0.01, 'threshold': 0.9676668873838575}}

In [11]:
threshold = optimizer.max["params"]["threshold"]
eps = optimizer.max["params"]["eps"]
X_hat = svt.svt_solve(X, mask, threshold=threshold, eps=eps, max_iters=10000)
X_hat

array([[0.22892789, 0.41564521, 0.11007147, 0.01646025, 0.10053958],
       [0.77715246, 0.25180181, 0.30136806, 0.08766734, 0.92679787],
       [0.08490553, 0.79136444, 0.61027389, 0.92797805, 0.76096421],
       [0.31477233, 0.52962952, 0.47664175, 0.71848776, 0.51068686],
       [0.38152334, 0.81824978, 0.21731755, 0.91290499, 0.87117235]])

not too far off!

In [7]:
X

array([[0.22892789, 0.41564521, 0.13619514, 0.01646025, 0.10053958],
       [0.77715246, 0.25180181, 0.23208707, 0.08766734, 0.92679787],
       [0.08490553, 0.79136444, 0.61027389, 0.92797805, 0.76096421],
       [0.57789451, 0.52962952, 0.47664175, 0.71848776, 0.51068686],
       [0.38152334, 0.81824978, 0.21731755, 0.91290499, 0.45441119]])